In [ ]:
from huggingface_hub import notebook_login


In [ ]:
notebook_login()

In [ ]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 12.7 MB/s eta 0:00:00


In [ ]:
pip install --upgrade transformers


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
import pandas as pd

In [ ]:
base_model = "meta-llama/Llama-2-7b-hf"
#Fine-tune model name

#Load the Dataset from hugging face
# dataset = load_dataset("ronitHF/pubmed-10k-8.1.1", split="train")
dataset=pd.read_csv('/content/drive/MyDrive/project/cleaned_train_data.csv')
val=pd.read_csv('/content/drive/MyDrive/project/cleaned_data.csv')
#Tokenizer
#Load the tokenizer from Llama 2
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
#In Llama2 we dont have the padding token which is a very big problem, because we have a dataset with different number of tokens in each row.
#So, we need to pad it so they all have the same length and here i am using end of sentence token and this will have an impact on the generation of our model
#I am using End of Sentence token for fine-tuning
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def count_words(text):
    return len(text.split())

In [ ]:
dataset2=dataset.dropna()
dataset2.isnull().sum()

article     0
abstract    0
dtype: int64

In [ ]:
word_count_article = dataset['article'].head(2).apply(count_words)
word_count_abstract = dataset['abstract'].head(2).apply(count_words)
word_count_article,word_count_abstract

(0    4614
 1    3135
 Name: article, dtype: int64,
 0    246
 1    353
 Name: abstract, dtype: int64)

In [ ]:
dataset2['article'] = dataset2['article'].apply(lambda x: ' '.join(x.split()[:4000]))

<ipython-input-23-4144af340a73>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset2['article'] = dataset2['article'].apply(lambda x: ' '.join(x.split()[:4000]))


In [ ]:
word_count_article = dataset2['article'].head(2).apply(count_words)
word_count_abstract = dataset2['abstract'].head(2).apply(count_words)
word_count_article,word_count_abstract

(0    4000
 1    3135
 Name: article, dtype: int64,
 0    246
 1    353
 Name: abstract, dtype: int64)

In [ ]:
val2=val.dropna()
val2.isnull().sum()

article     0
abstract    0
dtype: int64

In [ ]:
val2['article'] = val2['article'].apply(lambda x: ' '.join(x.split()[:4000]))

In [ ]:
dataset2 = dataset2.head(5000)
dataset2

,article,abstract
0,a recent systematic analysis showed that in 20...,background : the present study was carried ou...
1,it occurs in more than 50 of patients and may ...,backgroundanemia in patients with cancer who ...
2,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side eff...
3,"lepidoptera include agricultural pests that , ...",many lepidopteran insects are agricultural pe...
4,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough...
...,...,...
5133,intestinal malrotation is a congenital abnorma...,abstractmidgut volvulus is a highly life - th...
5134,the first two micrornas ( mirnas ) discovered ...,abstractthe discovery of micrornas ( mirnas )...
5135,tartrazine ( ta ) and sunset yellow ( sy ) are...,"we proposed a simple and effective method , b..."
5136,methicillin resistant staphylococcus aureus ( ...,community associated methicillin resistant st...


In [ ]:
val2 = val2.head(500)
val2

,article,abstract
0,"approximately , one - third of patients with s...",background and aim : there is lack of substan...
1,there is an epidemic of stroke in low and midd...,backgroundthe questionnaire for verifying str...
2,cardiovascular diseases account for the highes...,background : timely access to cardiovascular...
3,results of a liquid culturing system ( bd bact...,to determine differences in the ability of my...
4,the need for magnetic resonance imaging ( mri ...,aimsour aim was to evaluate the potential for...
...,...,...
495,"acquired hemophilia a ( aha ) , a rare bleedin...",we present a case report of a 52-year - old m...
496,the proband ( ii-1 ) was a 58-year - old man w...,objective : to identify the genetic defect fo...
497,tooth color is an important issue for esthetic...,objectives this in vitro study evaluated the ...
498,lead ( pb ) is a heavy metal found extensively...,in order to identify the presence of lead in ...


In [ ]:
#Configration of QLoRA
#Quantization Configuration
#To reduce the VRAM usage we will load the model in 4 bit precision and we will do quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #Quant type
    #We will use the "nf4" format this was introduced in the QLoRA paper
    bnb_4bit_quant_type="nf4",
    #As the model weights are stored using 4 bits and when we want to compute its only going to use 16 bits so we have more accuracy
    bnb_4bit_compute_dtype=torch.float16,
    #Quantization parameters are quantized
    bnb_4bit_use_double_quant=True,
)


# LoRA configuration
peft_config = LoraConfig(
    #Alpha is the strength of the adapters. In LoRA, instead of training all the weights, we will add some adapters in some layers and we will only
    #train the added weights
    #We can merge these adapters in some layers in a very weak way using very low value of alpha (using very little weight) or using a high value of alpha
    #(using a big weight)
    #15 is very big weight, usually 32 is considered as the standard value for this parameter
    lora_alpha=15,
    #10% dropout
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Cast the layernorm in fp32, make output embedding layer require grads, add the upcasting of the lmhead to fp32
#prepare_model_for_kbit_training---> This function basically helps to built the best model possible
model = prepare_model_for_kbit_training(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Set training arguments
training_arguments = TrainingArguments(
        output_dir="./",
        num_train_epochs=5,#3,5 good for the Llama 2 Model
        per_device_train_batch_size=2,# Number of batches that we are going to take for every step
        gradient_accumulation_steps=1,
        evaluation_strategy="steps",#Not helpful because we donot want to evaluate the model we just want to train it
        eval_steps=500,
        logging_steps=50,
        optim="paged_adamw_8bit",#Adam Optimizer we will be using but a version that is paged and in 8 bits, so it will lose less memory
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        warmup_steps=50,
        report_to="tensorboard",
        max_steps=-1, # if maximum steps=2, it will stop after two steps
)

In [ ]:
import datasets
from datasets import Dataset, DatasetDict



tds = Dataset.from_pandas(dataset2)
vds=Dataset.from_pandas(val2)



ds = DatasetDict()

ds['train'] = tds
ds['validation']=vds

In [ ]:


# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],#No separate evaluation dataset, i am using the same dataset
    peft_config=peft_config,
    dataset_text_field="article",
    max_seq_length=4000,# In dataset creation we put a threshold 2k for context length (input token limit) but we dont have enough VRAM unfortunately it will take a lot of VRAM to put everything into memory so we are just gonna stop at 512
    tokenizer=tokenizer,
    args=training_arguments,
    # Define your metric calculation function

)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 15.26 GiB. GPU 0 has a total capacty of 39.56 GiB of which 5.05 GiB is free. Process 13265 has 34.51 GiB memory in use. Of the allocated memory 22.56 GiB is allocated by PyTorch, and 11.45 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
drive_save_directory = "/content/drive/MyDrive/project/Trained model"

trainer.save_model(drive_save_directory)
tokenizer.save_pretrained(drive_save_directory)